<a href="https://colab.research.google.com/github/deanofthewebb/yolov7-1/blob/main/Yolov7_512_UnifiedNet_Export_and_Publish.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. Download Dependencies**

In [1]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


**Get the Latest Updated DeepSORT Tracking Script**

In [2]:
# !git clone https://github.com/ultralytics/yolov5.git
!git clone https://github.com/WongKinYiu/yolov7.git

Cloning into 'yolov7'...
remote: Enumerating objects: 231, done.
remote: Counting objects: 100% (66/66), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 231 (delta 38), reused 23 (delta 10), pack-reused 165
Receiving objects: 100% (231/231), 14.98 MiB | 25.82 MiB/s, done.
Resolving deltas: 100% (94/94), done.


# **2. Install Requirements**

In [ ]:
%cd yolov7
!pip install -r requirements.txt -qqq
!pip install onnx -qqq
!pip install onnxruntime-gpu -qqq
!pip install Pillow>=7.1.2 -qqq
!pip install awscli

In [ ]:
!aws configure

In [ ]:
# !mkdir /content/gdrive/MyDrive/Development/videos/3___yolov5m
# !aws s3 sync s3://vision-stream.livereachmedia.com/2BH2LRU3SXZJn4pQu8SNB6FI0zc/2022-07-01/ /content/gdrive/MyDrive/Development/videos/3___yolov5m/

In [ ]:
%cd /content/yolov7_deepsort/yolov7_deepsort/
import os
import sys
from pathlib import Path
from glob import glob
from PIL import Image
import shutil
import operator
import subprocess
import torch
from collections import OrderedDict
yolo_version = 'yolov7-e6e'
img_size = '1280'
yolo_alias = f'{yolo_version}_unifiednet_{img_size}'
# ckpt_path = f'/content/gdrive/MyDrive/checkpoints/vision-rx.yolov5m.inline.objects.unifiednet_512_beta_v2.ckpt'
ckpt_path = f'/content/gdrive/MyDrive/models/yolov7-e6e.pt'
# model = torch.hub.load('WongKinYiu/yolov7', yolo_version, autoshape=False, pretrained=False, classes=80, force_reload=True)
model = torch.load(ckpt_path)['model']

state_dict = model.state_dict()
print(state_dict.keys())

# revised_state_dict = OrderedDict([(k.replace('model.model', 'model'), v) if 'model.model' in k else (k, v) for k, v in state_dict.items()])
# revised_state_dict = OrderedDict([(k, v[:39]) if k in ['model.105.m.0.weight', 
#                                                       'model.105.m.0.bias',
#                                                       'model.105.m.1.weight',
#                                                       'model.105.m.1.bias',
#                                                       'model.105.m.2.weight',
#                                                       'model.105.m.2.bias',
#                                                        ] else (k, v) for k, v in revised_state_dict.items()])
# model.load_state_dict(state_dict, strict=True)

## **3. Run YOLOV5 Conversion**

In [ ]:

!pip install -qqq tensorflow torch torchvision umap-learn
!pip install -qqq ipywidgets>=7.5
!pip install -qqq opencv-python-headless==4.5.4.60
!pip install -qqq fiftyone

In [5]:
model.eval()
torch.save({'model': model, 'state_dict': state_dict, }, f'/content/{yolo_alias}.pt')

In [ ]:
import fiftyone as fo
from fiftyone import ViewField as F
import shutil
!cp -r /content/gdrive/MyDrive/configs/.aws /content/
!chmod -R 777 /content/.aws
%cd /content/yolov7_deepsort/
!python setup.py install
%cd /content/yolov7_deepsort/yolov7_deepsort/
encoding = 'utf-8'
s3_upload_path = 's3://data-labeling.livereachmedia.com/datasets/unifiednet/distilled_yolov7/'
def run_patch_extraction_and_annotation(args):
    def subprocess_run(cmd):
        p = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE,
                            stderr=subprocess.STDOUT)
        stdout = []
        while True:
            line = p.stdout.readline()
            stdout.append(line.decode(encoding))
            print(line.decode(encoding))
            if line.decode(encoding) == '' and p.poll() != None:
                break
        return ''.join(stdout)

    processed_dir = args['processed_dir']
    for device in args['devices']:
        # Process Folder #
        for path, dirs, files in os.walk(f"/content/gdrive/MyDrive/videos/{args['devices'][0]}/"):
            for dir in reversed(dirs):
                print("VIDEO DIR",args['videos_dir'].as_posix() )
                for file_ in Path(path+'/'+dir).iterdir():
                    new_save_dir = args['save_dir'] / Path(file_).stem
                    os.makedirs(new_save_dir, exist_ok=True)

                    tmp_save_dir = Path('/content/inference') / Path(file_).stem
                    os.makedirs(tmp_save_dir, exist_ok=True)
                    old_video_path = file_
                    # print(old_video_path)
                    video_processed_fpath = args['processed_dir'] / Path(file_).stem
                    # print(video_processed_fpath)
                    # Run Detection
                    print("Running YOLO")
                    cmd = f"python detect_track_trails.py  \
                                  --class 0 2 5 7 \
                                  --agnostic-nms \
                                  --source {os.path.abspath(file_)} \
                                  --output {tmp_save_dir.as_posix()} \
                                  --weights /content/yolov7_deepsort/yolov7-e6e_unifiednet_1280.pt \
                                  --conf 0.35 \
                                  --cfg /content/yolov7_deepsort/yolov7_deepsort/cfg/deploy/yolov7-e6e.yaml \
                                  --img-size {img_size} --device 0"
                    subprocess_run(cmd)
                    !aws s3 sync /content/inference/{Path(file_).stem}/datasets {s3_upload_path}
                    shutil.move(tmp_save_dir.as_posix(),new_save_dir.as_posix())
                    

if __name__ == '__main__':
    args = { 'save_dir': Path('/content/gdrive/MyDrive/inference/'),
                    
                    'devices': ['yolov7'],
                    'videos_dir': Path('/content/gdrive/MyDrive/videos/'),
                    'processed_dir': Path('/content/gdrive/MyDrive/videos/raw_videos_detected/')}
    run_patch_extraction_and_annotation(args)

/content/yolov7_deepsort
/usr/local/lib/python3.7/dist-packages/setuptools/dist.py:484: UserWarning: Normalizing '0.01' to '0.1'
  warnings.warn(tmpl.format(**locals()))
running install
running bdist_egg
running egg_info
writing Keystone_Pipeline.egg-info/PKG-INFO
writing dependency_links to Keystone_Pipeline.egg-info/dependency_links.txt
writing top-level names to Keystone_Pipeline.egg-info/top_level.txt
writing manifest file 'Keystone_Pipeline.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
copying keystone/fiftyone_custom_documents.py -> build/lib/keystone
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/keystone
copying build/lib/keystone/__init__.py -> build/bdist.linux-x86_64/egg/keystone
copying build/lib/keystone/fiftyone_custom_documents.py -> build/bdist.linux-x86_64/egg/keystone
byte-compiling build/bdist.linux-x86_64/egg/keystone/__init__.py to __init__.cpython-37.pyc
byte-compilin

In [ ]:
model.eval()
torch.save(model, f'/content/{yolo_alias}_pre-wts.pt')
%cd /content/yolov7/
# !cp -ur /content/gdrive/MyDrive/scripts/gen_wts_yoloV5.py .
yolo_path = f"{yolo_alias}_pre-wts.pt"
# !cp -ur /content/gdrive/MyDrive/configs/{yolo_version}.yaml .

!python gen_wts_yoloV5.py -w {yolo_path} --yaml /content/yolov7/cfg/deploy/yolov7-e6e.yaml -mw {img_size} -mh {img_size}
# !mkdir /content/gdrive/MyDrive/weights/exported_weights/{yolo_version}_{img_size}/{yolo_alias}_export/
# !cp -r /content/{yolo_alias}.pt /content/gdrive/MyDrive/weights/exported_weights/{yolo_version}_{img_size}/{yolo_alias}_export/
# !cp -r /content/{yolo_alias}_pre-wts.pt /content/gdrive/MyDrive/weights/exported_weights/{yolo_version}_{img_size}/{yolo_alias}_export/{yolo_alias}.pt
# !cp -r /content/{yolo_alias}_pre-wts.wts /content/gdrive/MyDrive/weights/exported_weights/{yolo_version}_{img_size}/{yolo_alias}_export/{yolo_alias}.wts
# !cp -r /content/{yolo_alias}_pre-wts.cfg /content/gdrive/MyDrive/weights/exported_weights/{yolo_version}_{img_size}/{yolo_alias}_export/{yolo_alias}.cfg


In [ ]:
# !cat /content/gdrive/MyDrive/.aws/credentials
!pip install -U PyYAML


In [ ]:
# !pip install awscli && aws configure

In [ ]:
!cp /content/yolov7.onnx /content/vision-rx.yolov7.inline.objects.coco_640.onnx
# !cp /content/yolov7-tiny.cfg vision-rx.yolov7.inline.objects.coco_416.cfg 

In [ ]:
!aws s3 cp 

In [ ]:
!python models/export.py --weights /content/yolov7.pt --grid

In [ ]:
%cd /content/yolov7/
!aws s3 cp --metadata parameters-config=1 vision-rx.yolov7-tiny.inline.objects.coco_416.wts s3://ml.livereachmedia.com/darknet/inline/yolov7-tiny/vision-rx.yolov7-tiny.inline.objects.coco_416.wts


In [ ]:
!aws s3 cp --metadata parameters-config=1 /content/yolov7.onnx   s3://ml.livereachmedia.com/darknet/inline/yolov7/vision-rx.yolov7.inline.objects.coco_640.onnx


In [ ]:
!aws s3 cp s3://data-labeling.livereachmedia.com/datasets/unifiednet/yolor_deepsort.zip . && unzip -qqq yolor_deepsort.zip